## Table 8 Average Adjustment in Actual and Preferred Hours When Developing and Resolving Working Hour Mismatches


i) When people go from wanting to work more/less at t-1 to being content with their hours at time t. 
First two rows. 

ii) Those who report a satisfaction at time t-1 and then report dissatisaction of either type (more/less hours) at time t. 
Rows 3 and 4.

- Is it working hours that change or is it preferred hours?

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import numbers

In [185]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
filenames = glob('Combined_*[a,b,c,d,e,f,g,h,i,j,k]*160c.dta')
list_dfs  = [pd.read_stata(f) for f in filenames]

In [226]:
male, female = '[1] Male', '[2] Female'
less, more, same = '[1] Fewer hours', '[3] More hours', '[2] About the same'

In [315]:
def table_8(sex, pref_before, pref_now):
    
    list_males = [list_dfs[i][list_dfs[i].filter(regex='hgsex$').columns]for i in range(11)]
    list_males = [list_dfs[i][list_males[i].iloc[:,0] == sex] for i in range(11)]
            
    pref1 = [list_males[i].filter(regex='jbhrcpr$') for i in range(11)]
    pref2 = [list_males[i].filter(regex='jbhrcpr$') for i in range(11)]
    pref2 = pref2[1:]

    diff = [pref1[i].join(pref2[i]) for i in range(10)]
    cleaned = [diff[i].dropna(how = 'any') for i in range(10)]

    hours_wkedt = [list_dfs[i][list_dfs[i].filter(regex='jbhruc$').columns]for i in range(11)]
    hours_wkednew = hours_wkedt[1:]
    hours_preferred = [list_dfs[i][list_dfs[i].filter(regex='jbtprhr$').columns]for i in range(11)]
    hours_preferred_new = hours_preferred[1:]

    new_clean = [cleaned[i].join(hours_wkedt[i]) for i in range(10)]
    new_clean = [new_clean[i].join(hours_wkednew[i]) for i in range(10)]
    new_clean = [new_clean[i].join(hours_preferred[i]) for i in range(10)]
    new_clean = [new_clean[i].join(hours_preferred_new[i]) for i in range(10)]

    fewer_at_tminus1 = [new_clean[i][new_clean[i].iloc[:,0] == pref_before] for i in range(10)]
    fewer_at_tminus1 = [fewer_at_tminus1[i][fewer_at_tminus1[i].iloc[:,1] == pref_now] for i in range(10)]
    fewer_at_tminus_pref = [pd.to_numeric(fewer_at_tminus1[i].iloc[:,-2], errors='coerce') for i in range(10)]
    fewer_at_tminus_actual = [pd.to_numeric(fewer_at_tminus1[i].iloc[:,2], errors='coerce') for i in range(10)]

    fewer_at_t_pref = [pd.to_numeric(fewer_at_tminus1[i].iloc[:,-1], errors='coerce') for i in range(10)]
    fewer_at_t_actual = [pd.to_numeric(fewer_at_tminus1[i].iloc[:,3], errors='coerce') for i in range(10)]

    hours_diff_pref = [fewer_at_t_pref[i] - fewer_at_tminus_pref[i] for i in range(10)]
    hours_diff_actual = [fewer_at_t_actual[i] - fewer_at_tminus_actual[i] for i in range(10)]

    hours_diff_pref = [pd.DataFrame(hours_diff_pref[i]) for i in range(10)]
    hours_diff_pref = [hours_diff_pref[i].fillna(0) for i in range(10)]
    hours_diff_actual = [pd.DataFrame(hours_diff_actual[i]) for i in range(10)]
    hours_diff_actual = [hours_diff_actual[i].fillna(0) for i in range(10)]

    for i in range(10):
        hours_diff_actual[i].columns = {'Actual hours'}

    final = [fewer_at_tminus1[i].join(hours_diff_pref[i]) for i in range(10)]
    final = [final[i].join(hours_diff_actual[i]) for i in range(10)]
    actual_diff = [final[i].iloc[:,-1] for i in range(10)]
    actual_diff_mean = [actual_diff[i].mean() for i in range(10)]
    actual_diff_mean = np.array(actual_diff_mean).mean()
    
    pref_diff = [final[i].iloc[:,-2] for i in range(10)]
    pref_diff_mean = [pref_diff[i].mean() for i in range(10)]
    pref_diff_mean = np.array(pref_diff_mean).mean()
    
    add = [len(final[i]) for i in range(10)]
    add = np.array(add).sum()
   

    return actual_diff_mean, pref_diff_mean, add



In [319]:
m_l_s = table_8(male,less, same) #Resolved desire for less
m_m_s = table_8(male,more, same)
f_l_s = table_8(female,less, same)
f_m_s = table_8(female, more, same)
m_s_l = table_8(male, same, less) 
m_s_m = table_8(male, same, more)
f_s_l = table_8(female, same, less)
f_s_m = table_8(female, same, more)

In [352]:
actual_male = m_s_l[0], m_s_m[0], m_l_s[0], m_m_s[0]
preferred_male = m_s_l[1], m_s_m[1], m_l_s[1], m_m_s[1]
total_male = m_s_l[2], m_s_m[2], m_l_s[2], m_m_s[2]

actual_female = f_s_l[0], f_s_m[0], f_l_s[0], f_m_s[0]
preferred_female = f_s_l[1], f_s_m[1], f_l_s[1], f_m_s[1]
total_female = f_s_l[2], f_s_m[2], f_l_s[2], f_m_s[2]

In [364]:
males = pd.DataFrame(preferred_male, actual_male)
males = males.reset_index()
total_male = pd.DataFrame(total_male)
total_male.columns = ['Number of Workers (M)']
males = males.join(total_male)
males.columns = [['Change in Actual hours (M)', 'Change in Preferred hours (M)', 'Number of Workers (M)']]

females = pd.DataFrame(preferred_female, actual_female)
females = females.reset_index()
total_female = pd.DataFrame(total_female)
total_female.columns = ['Number of Workers (F)']
females = females.join(total_female)
females.columns = [['Change in Actual hours (F)', 'Change in Preferred hours (F)', 'Number of Workers (F)']]
final = males.join(females)

In [371]:
final.index = ('Developed desire for fewer hours',
'Developed desire for more hours',
'Resolved a desire for fewer hours',
'Resolved a desire for more hours')

In [372]:
final

,Change in Actual hours (M),Change in Preferred hours (M),Number of Workers (M),Change in Actual hours (F),Change in Preferred hours (F),Number of Workers (F)
Developed desire for fewer hours,4.030498,-8.843235,3984,4.420654,-7.336424,3388
Developed desire for more hours,-4.600679,6.377284,1805,-3.866546,6.064928,2018
Resolved a desire for fewer hours,-4.552529,8.561799,3868,-5.143162,6.592495,3172
Resolved a desire for more hours,7.746297,-3.211428,2180,7.778065,-2.349243,2316
